<a href="https://colab.research.google.com/github/minomugeshi/AddWatermarkForPDF/blob/main/GoogleColab%E3%82%A6%E3%82%A9%E3%83%BC%E3%82%BF%E3%83%BC%E3%83%9E%E3%83%BC%E3%82%AF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PDFウォーターマーク印字
コードの目的:
このコードは、Google Colab 上で実行できる Python プログラムで、PDF ファイルにウォーターマークを追加し、処理されたファイルの一覧を記載した履歴ファイルを作成する機能を提供します。

In [ ]:
# @title ウォーターマーク印字

# インストールが必要なライブラリ
print("初期化")
try:
  import reportlab, PyPDF2, fpdf
except ImportError:
  !pip install reportlab PyPDF2 fpdf

import reportlab
import PyPDF2

import io
from IPython.display import display, HTML
from ipywidgets import widgets, VBox, Output
from fpdf import FPDF
from tqdm.notebook import tqdm #プログレス表示の追加
from IPython.display import clear_output
from datetime import datetime, timezone, timedelta
from google.colab import files, drive
from googleapiclient.discovery import build
import tempfile
from reportlab.platypus import SimpleDocTemplate, BaseDocTemplate, PageTemplate, Frame, Table, TableStyle, PageBreak, Paragraph, KeepTogether, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import mm
from reportlab.lib import colors
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
import os
import time

import subprocess
# フォントのインストール
subprocess.run(["apt-get", "install", "-y", "fonts-ipafont-gothic"])

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Define directories
input_dir = '/content/drive/MyDrive/出図IN'
output_dir = '/content/drive/MyDrive/出図OUT'

# Ensure output directory exists
#os.makedirs(output_dir, exist_ok=True)

# Step 3: Register fonts
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

# フォントファイルのパス (ローカルパス)
font_path = '/usr/share/fonts/opentype/ipafont-gothic/ipag.ttf' # IPAゴシックフォントのパス

# フォントの登録
pdfmetrics.registerFont(TTFont('IPAGothic', font_path))

def is_pdf_editable(pdf_path):
    try:
        with open(pdf_path, "rb") as file:
            reader = PyPDF2.PdfReader(file)
            if reader.is_encrypted:
                return False  # 暗号化されたPDFは編集不可とみなす
            writer = PyPDF2.PdfWriter()
            writer.append_pages_from_reader(reader)
            # テスト用の一時ファイルに書き込みを試みる
            with tempfile.NamedTemporaryFile(delete=False) as temp_file:
                writer.write(temp_file)
            os.unlink(temp_file.name)  # テスト用ファイルを削除
            return True
    except Exception as e:
        print(f"Error checking PDF editability ({pdf_path}): {str(e)}")
        return False

## Function to list PDF files
#def list_pdf_files():
#    return [f for f in os.listdir(input_dir) if f.endswith('.pdf')]

def list_pdf_files():
    editable_pdfs = []
    for f in os.listdir(input_dir):
        if f.endswith('.pdf'):
            pdf_path = os.path.join(input_dir, f)
            if is_pdf_editable(pdf_path):
                editable_pdfs.append(f)
            else:
                print(f"スキップされました (編集不可): {f}")
    return editable_pdfs

def add_watermark(pdf_path, watermark_text):
    try:
        with open(pdf_path, "rb") as file:
            reader = PyPDF2.PdfReader(file)
            writer = PyPDF2.PdfWriter()

            page_count = len(reader.pages)

            for page_number in range(page_count):
                page = reader.pages[page_number]

                # ページの幅と高さを取得
                page_width = float(page.mediabox.width)
                page_height = float(page.mediabox.height)

                # キャンバスを作成してウォーターマークを追加
                packet = io.BytesIO()
                can = canvas.Canvas(packet, pagesize=(page_width, page_height))

                # ウォーターマークのテキストを描画
                current_time = get_japan_time().strftime('%Y-%m-%d %H:%M:%S')
                combined_text = f"{watermark_text} - {current_time} - {employee_id_widget.value}"

                # ウォーターマークのフォントサイズを設定
                font_size = min(page_width, page_height) * 0.015  # 用紙幅の2%をフォントサイズとして設定
                can.setFont("IPAGothic", font_size)

                # ウォーターマークの文字列の幅と高さを取得
                text_width = can.stringWidth(combined_text, "IPAGothic", font_size)
                text_height = font_size  # フォントの高さとして使用

                # ウォーターマークの位置を計算
                x = page_width - 1.5*mm - text_width
                y = page_height - 0.5*mm - text_height  # 上端からの距離

                can.setFillColorRGB(0.5, 0.5, 0.5, alpha=0.5)
                can.drawString(x, y, combined_text)
                can.save()

                packet.seek(0)
                watermark = PyPDF2.PdfReader(packet)
                page.merge_page(watermark.pages[0])
                writer.add_page(page)

                # メモリを解放
                page = None

            output_filename = f"({watermark_text}){os.path.basename(pdf_path)}"
            output_path = os.path.join(output_dir, output_filename)
            with open(output_path, "wb") as output_stream:
                writer.write(output_stream)

            # メモリを解放
            reader = None
            writer = None

        return output_filename, page_count

    #エラーハンドリングの強化
    except PyPDF2.errors.PdfReadError as e:
        print(f"PDFファイルの読み込みエラー ({pdf_path}): {str(e)}")
    except PermissionError as e:
        print(f"ファイルアクセス権限エラー ({pdf_path}): {str(e)}")
    except Exception as e:
        print(f"予期しないエラーが発生しました ({pdf_path}): {str(e)}")
    return None, 0

# UI elements
output = Output()

# 社員番号入力
employee_id_widget = widgets.Text(description='社員番号:')
load_button = widgets.Button(description="読み込み", disabled=True)

def on_employee_id_change(change):
    if employee_id_widget.value:
        load_button.disabled = False
    else:
        load_button.disabled = True

employee_id_widget.observe(on_employee_id_change, names='value')

def on_load_button_clicked(b):
    with output:
        output.clear_output()

        employee_id = employee_id_widget.value
        if not employee_id:
            print("社員番号を入力してください。")
            return

        # Google Drive のマイドライブ直下に「出図IN」フォルダと「出図OUT」フォルダの有無を確認
        if not os.path.exists(input_dir) or not os.path.exists(output_dir):
            print('マイドライブ直下に「出図IN」フォルダと「出図OUT」フォルダを作成してください。')
            return  # フォルダがない場合は処理を終了

        # フォルダ作成後、PDFファイルの確認処理を実行
        pdf_files = list_pdf_files()
        if pdf_files:
            print(f"編集可能なPDFファイルが {len(pdf_files)} 件見つかりました:")
            total_page_count = 0
            for i, pdf in enumerate(pdf_files, start=1):
                pdf_path = os.path.join(input_dir, pdf)
                try:
                    with open(pdf_path, "rb") as f:
                        reader = PyPDF2.PdfReader(f)
                        page_count = len(reader.pages)
                        print(f"{i}. {pdf} (枚数: {page_count})")
                        total_page_count += page_count
                except Exception as e:
                    print(f"Error reading {pdf}: {e}")
            print(f"合計枚数: {total_page_count}")
            show_all_select_button()
        else:
            print("編集可能なPDFファイルが「出図IN」フォルダ内に見つかりませんでした。")

def show_all_select_button():
    all_select_button = widgets.Button(description="全選択")

    def on_all_select_button_clicked(b):
        with output:
            output.clear_output()
            selected_pdfs = list_pdf_files()
            print(f"Selected {len(selected_pdfs)} PDF files.")
            show_watermark_input(selected_pdfs)

    all_select_button.on_click(on_all_select_button_clicked)
    display(all_select_button)

def show_watermark_input(selected_pdfs):
    watermark_text_widget = widgets.Text(
        description='Watermark:',
        disabled=False,
        max_length=20  # 入力可能な文字数を20に制限
    )
    confirm_button = widgets.Button(description="印字")

    def on_confirm_button_clicked(b):
        time.sleep(0.5)  # 0.5秒のディレーを追加
        global watermark_text
        watermark_text = watermark_text_widget.value

        if len(watermark_text) > 20:
            with output:
                output.clear_output()
                print("Error: ウォーターマークは20文字までです。")
                show_watermark_input(selected_pdfs)
        else:
            with output:
                output.clear_output()
                # print の出力を output オブジェクトにリダイレクト
                with output:
                    print(f"入力されたウォーターマーク: {watermark_text}")
                yes_button = widgets.Button(description="YES")
                no_button = widgets.Button(description="NO")

                def on_yes_button_clicked(yes_b):
                    with output:
                        output.clear_output()
                        print(f"印字処理( {watermark_text} )")

                        def process_pdf_files(selected_pdfs, watermark_text):
                            watermarked_files = []
                            total_page_count = 0
                            total_files = len(selected_pdfs)

                            # tqdmを使用してプログレスバーを作成
                            with tqdm(total=total_files, desc="PDFファイルを処理中", unit="file") as pbar:
                                for pdf in selected_pdfs:
                                    pdf_path = os.path.join(input_dir, pdf)
                                    try:
                                        output_filename, page_count = add_watermark(pdf_path, watermark_text)
                                        if output_filename:
                                            watermarked_files.append((output_filename, page_count))
                                            total_page_count += page_count

                                            # プログレスバーを更新し、現在のファイル情報を表示
                                            pbar.set_postfix_str(f"完了: {pdf} (枚数: {page_count})")
                                            pbar.update(1)
                                        else:
                                            tqdm.write(f"スキップ: {pdf} (ウォーターマーク追加に失敗)")
                                    except Exception as e:
                                        tqdm.write(f"エラー: {pdf} の処理中に問題が発生しました - {str(e)}")

                            return watermarked_files, total_page_count

                        watermarked_files, total_page_count = process_pdf_files(selected_pdfs, watermark_text)

                        print(f"\nウォーターマーク処理が完了しました。")
                        print(f"処理したファイル数: {len(watermarked_files)}")
                        print(f"合計枚数: {total_page_count}")
                        if watermarked_files:
                            print("\n処理済みファイル一覧:")
                            for file, page_count in watermarked_files:
                                print(f"{file} (枚数: {page_count})")
                        else:
                            print("\n正常に処理されたファイルはありません。")

                        # リストPDFの作成とダウンロード
                        if watermarked_files:
                            create_and_download_list_pdf(watermark_text, watermarked_files)
                        else:
                            print("\nリストPDFは作成されません。処理済みファイルがありません。")

                def on_no_button_clicked(no_b):
                    with output:
                        output.clear_output()
                        show_watermark_input(selected_pdfs)

                yes_button.on_click(on_yes_button_clicked)
                no_button.on_click(on_no_button_clicked)
                display(VBox([widgets.Label(value='これでOKですか？'), yes_button, no_button]))

    confirm_button.on_click(on_confirm_button_clicked)
    display(VBox([watermark_text_widget, confirm_button]))

# 現在の日本時間を取得する関数
def get_japan_time():
    JST = timezone(timedelta(hours=+9), 'JST')  # 日本時間のタイムゾーン
    return datetime.now(JST)

class CustomCanvas(canvas.Canvas):
    def __init__(self, *args, **kwargs):
        self.watermark_text = kwargs.pop('watermark_text', '')
        self.total_pages = kwargs.pop('total_pages', 0)
        canvas.Canvas.__init__(self, *args, **kwargs)
        self._saved_page_states = []

    def showPage(self):
        self._saved_page_states.append(dict(self.__dict__))
        self._startPage()

    def save(self):
        num_pages = len(self._saved_page_states)
        for state in self._saved_page_states:
            self.__dict__.update(state)
            self.draw_page_info(num_pages)
            canvas.Canvas.showPage(self)
        canvas.Canvas.save(self)

    def draw_page_info(self, page_count):
        self.setFont("IPAGothic", 9)
        current_time = get_japan_time().strftime('%Y-%m-%d %H:%M:%S')

        # ヘッダー情報を描画
        self.draw_info(287*mm)  # ページ上部

        # フッター情報を描画
        self.draw_info(10*mm)   # ページ下部

    def draw_info(self, y_position):
        # 現在の日時を取得
        current_time = get_japan_time().strftime('%Y-%m-%d %H:%M:%S')

        # 左側に watermark_text と日時を表示
        left_text = f"{self.watermark_text} - {current_time} - {employee_id_widget.value}"
        self.drawString(10*mm, y_position, left_text)

        # 中央に合計枚数を表示
        center_text = f"合計枚数: {self.total_pages}"
        self.drawCentredString(105*mm, y_position, center_text)

        # 右側にページ番号/総ページ数を表示
        right_text = f"({self._pageNumber}/{len(self._saved_page_states)})"
        self.drawRightString(200*mm, y_position, right_text)

def create_and_download_list_pdf(watermark_text, watermarked_files):

    # A4サイズの定義
    page_width, page_height = A4

    # 余白の設定（ヘッダーとフッターのスペースを確保）
    left_margin = right_margin = 10*mm
    top_margin = bottom_margin = 10*mm

    # 出力ファイル名の設定
    output_filename = f"{watermark_text}_list_{employee_id_widget.value}.pdf"

    # 出力ファイルのフルパスを設定（Google Driveのルートディレクトリ）
    output_path = '/content/drive/My Drive/' + output_filename

    # ドキュメントの作成
    doc = SimpleDocTemplate(
        output_path,
        pagesize=A4,
        leftMargin=left_margin,
        rightMargin=right_margin,
        topMargin=top_margin,
        bottomMargin=bottom_margin
    )

    # テーブルデータの作成
    data = [["No.", "ファイル名", "枚数"]]
    for i, (file, page_count) in enumerate(watermarked_files, start=1):
        data.append([str(i), file, str(page_count)])

    # 合計枚数の計算
    total_pages = sum(page_count for _, page_count in watermarked_files)

    # テーブルスタイルの設定
    style = TableStyle([
        ('BACKGROUND', (0,0), (-1,0), colors.grey),
        ('TEXTCOLOR',(0,0),(-1,0),colors.whitesmoke),
        ('ALIGN',(0,0),(-1,-1),'CENTER'),
        ('FONTNAME', (0,0), (-1,-1), 'IPAGothic'),
        ('FONTSIZE', (0,0), (-1,0), 12),
        ('FONTSIZE', (0,1), (-1,-1), 10),
        ('BOTTOMPADDING', (0,0), (-1,0), 6),
        ('BACKGROUND',(0,1),(-1,-1),colors.beige),
        ('TEXTCOLOR',(0,1),(-1,-1),colors.black),
        ('ALIGN', (1,1), (1,-1), 'LEFT'),
        ('TOPPADDING', (0,1), (-1,-1), 3),
        ('BOTTOMPADDING', (0,1), (-1,-1), 3),
        ('GRID', (0,0), (-1,-1), 1, colors.black),
    ])

    # ドキュメントの構築
    elements = []

    # テーブルを40行ずつに分割
    for i in range(0, len(data), 40):
        table_data = data[0:1] + data[i+1:i+41]  # ヘッダー行 + 40行のデータ
        table = Table(table_data, colWidths=[30, page_width - left_margin - right_margin - 80, 50])
        table.setStyle(style)
        elements.append(table)
        if i + 40 < len(data):  # 最後のページ以外にPageBreakを追加
            elements.append(PageBreak())

    # PDFの生成
    doc.build(elements, canvasmaker=lambda *args, **kwargs: CustomCanvas(*args, **kwargs, watermark_text=watermark_text, total_pages=total_pages))

    print(f"履歴ファイル: {output_filename} Google Driveのルートに保存済み。")

    display_fixed_folder_link()

    time.sleep(0.5)  # 0.5秒のディレーを追加
    # ファイルのダウンロード
    files.download(output_path)

def display_fixed_folder_link():
    folder_name = '出図OUT'
    folder_link = output_dir

    print(f"'{folder_name}' フォルダからダウンロードしてください：")

# Display load button
#display(employee_id_widget)
display(employee_id_widget, load_button, output)
load_button.on_click(on_load_button_clicked)


初期化
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 8.9 MB/s eta 0:00:00
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=d063042e2355b3d752d49e81facf5f7ecd3a79edf607435c465abccbea2e484f
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf
Mounted at /content/drive


Text(value='', description='社員番号:')

Button(description='読み込み', disabled=True, style=ButtonStyle())

Output()